In [81]:
import gradio as gr
import requests

# Azure OpenAI 설정
AZURE_OPENAI_ENDPOINT = 'https://1507-ai.openai.azure.com/'
AZURE_OPENAI_API_KEY = '22646ec32d23457ca3ed253eefc5342f'
DEPLOYMENT_NAME = '1507-gpt4o'

# 지역 목록
regions = [
    "원하는 지역을 선택해주세요.",
    "서울", "부산", "인천", "대구", "대전", "광주", "울산", "세종",
    "경기", "충북", "충남", "전남", "전북", "경북", "경남", "강원", "제주"
]

# 지역별 이미지 URL 맵핑
region_images = {
    "서울": "https://blog.kakaocdn.net/dn/sQ19E/btqwvLFwzLq/E4h6Km5fbU479eiEJ3AVn0/img.jpg",
    "부산": "https://biz.chosun.com/resizer/rI3f1mbX9aVT3qCq5Ouc5H60j7g=/616x0/smart/cloudfront-ap-northeast-1.images.arcpublishing.com/chosunbiz/STFOCUYQQFHLHOOTRPSNIMCJWA.png",
    "인천": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTuIgKcIFU8HJ5y0aChkY4fFDoaVvLgvCHWvQ&s",
    "대구": "https://www.buk.daegu.kr/cmsh/buk.daegu.kr/images/introduction/mark.jpg",
    "대전": "https://pimg.mk.co.kr/meet/neds/2022/08/image_readtop_2022_739002_16611084625143117.jpg",
    "광주": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTGEaZgAZK7-EyCoeVecnY62qOmAW1WLHkFvA&s",
    "울산": "https://i.namu.wiki/i/t0Y4yIdyR1XRvidCKQgwKdirJoMHUy7Eobst3LZFq7_IazAI8rgxqrX3ANkZeKCsVcpz2teRbSL8AIJDAUWU-A.svg",
    "세종": "https://i.namu.wiki/i/bY8VhFcZR9v2rHasHzaiqe9cBhLembRbL2jUE8wLJ_mGmsGYWJPdrjZ_9zZRp7iOP9lkh8pH2E59_V1qgULz-Q.svg",
    "경기": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2W52tgxbrS6gsvYPX0j0kj0eswn_W1NPOgA&s",
    "충북": "https://i.namu.wiki/i/wzg69LiSxv7cvzpwvUtQsLfXQw7kuypTkQrTYF8SXs--s9ms-AuYov8Ygs42Owz-vzxQ3ZbGewUosH60KRz4ig.webp",
    "충남": "https://i.namu.wiki/i/6XRo5QjySVA44vwpSHTXuX3pmktcD2iVyXr7qY9tUCVLrFvygZrEE1azM2KE5Xx7PE8LCtnJvFtS4U2LMILiVA.svg",
    "전남": "https://i.namu.wiki/i/UobFnXCPHhNqhhjjtkSkSiSy-ykC71kOwMMVr6X0zhEEOOrVLmWUKZFbfxR1qj3HNP1bUsv_4vYC6IMD7Bi9Cg.svg",
    "전북": "https://i.namu.wiki/i/lDxAer6TneqL9KBome6tnOYqspnC_BxehDornzGfDt6_zJd967mOulVThcivihY5MxfZeHn2OGFluolNpB3fUw.svg",
    "경북": "https://i.namu.wiki/i/qXW_JKmivagUNdTCoN3XTonSVHHNBHHE8J5W2i-D5fbBQiZvdqavC6z1ePPW6lYgfGLUPZy7yW6Mx-JPDyNTZA.webp",
    "경남": "https://i.namu.wiki/i/-N8i4lAMMHqSJOYtiRN_S10unCPn9Bzc9mAMToo85eBSlaIKJ4Ybb2e8cY8jhoEPC_Ln9NMv71cEhsdARSNIbQ.webp",
    "강원": "https://state.gwd.go.kr/page/gwprovin/images/main/logo_b.png?ver=20231204",
    "제주": "https://image.utoimage.com/preview/cp872655/2018/11/201811011845_500.jpg",
    "원하는 지역을 선택해주세요.": "https://cdn.inflearn.com/public/files/blogs/35722e4a-e052-49cb-a621-2a9b63c7666c/%EA%B3%A0%EC%96%91%EC%9D%B4.png",
}

# Azure OpenAI API 호출 함수  
def chatgpt_response(prompt, region, history):
    # 지역이 "원하는 지역을 선택해주세요."일 경우 경고 메시지 출력
    if region == "원하는 지역을 선택해주세요.":
        bot_response = "원하는 지역을 선택 후 재실행 해주세요."
        history.append((prompt, bot_response))
        image_url = region_images.get(region)
        return '', history, image_url

    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    messages = []

    # System
    system_message = f"너는 {region}의 관광지에 대한 전문가야. 유저가 작성한 장소에서 어느 곳이 가까운지, 또 {region}에 있는 관광지에 대한 정보를 알려줘"
    messages.append({"role": "system", "content": system_message})

    # assistant
    if len(history) > 0:
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

    # user
    messages.append({"role": "user", "content": prompt})

    payload = {
        "messages": messages,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 4096
    }

    response = requests.post(  
        f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",  
        headers=headers,  
        json=payload  
    )  
    
    result = response.json()  
    bot_response = result['choices'][0]['message']['content'].strip()
    history.append((prompt, bot_response))  
    
    # 선택된 지역에 따른 이미지 URL
    image_url = region_images.get(region)
    
    return '', history, image_url

# Gradio 인터페이스
with gr.Blocks(css=".title {text-align: center; color: #f5f0f0; font-family: 'Arial';} .footer {text-align: right; color: #7f8c8d; font-family: 'Arial';}") as demo:
    gr.Markdown("<h1 class='title'>ChatGPT RestSpace Demo</h1>")
    
    with gr.Column():
        region_dropdown = gr.Dropdown(choices=regions, label="지역 선택", value="원하는 지역을 선택해주세요.")
        with gr.Row():
            user_input = gr.Textbox(show_label=False, placeholder="Type your message here...", scale=8)
            submit_btn = gr.Button("Send", variant="primary", scale=2)
            clear_btn = gr.Button("Clear", variant="secondary")
    chatbot = gr.Chatbot()
    image_output = gr.Image(width=1024, height=1024)

    gr.Markdown("<h3 class='footer'>제작자: CheongYa</h3>")
    gr.Markdown("<h3 class='footer'>최종 수정일: 2024-08-02</h3>")

    # 버튼 클릭 시 이벤트 처리
    submit_btn.click(chatgpt_response, [user_input, region_dropdown, chatbot], [user_input, chatbot, image_output])
    clear_btn.click(lambda: [None, None], None, [chatbot, image_output], queue=False)
    # clear_btn.click(lambda: None, None, image_output, queue=False)
    user_input.submit(chatgpt_response, [user_input, region_dropdown, chatbot], [user_input, chatbot, image_output])

# 인터페이스 실행
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7946

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
